In [7]:
import math
from PIL import Image, ImageDraw

def ensure_clockwise_quad(polygon):
    """
    Takes a list of 4 (x, y) points and ensures they are ordered clockwise.
    Returns a new list of points in clockwise order.
    """
    if len(polygon) != 4:
        raise ValueError("Expected exactly 4 points.")

    # centroid
    cx = sum(p[0] for p in polygon) / 4
    cy = sum(p[1] for p in polygon) / 4

    # angles from centroid
    def angle_from_centroid(pt):
        return (math.atan2(pt[1] - cy, pt[0] - cx) + 2 * math.pi) % (2 * math.pi)

    # sort  (counter-clockwise)
    sorted_pts = sorted(polygon, key=angle_from_centroid)

    # return clockwise
    return sorted_pts[::-1]


def polygon_to_yolo_label_from_image_dict(image: Image.Image, data: dict, class_id=0) -> str:
    """
    Converts the 'runwayLabel' polygon from the data dict into YOLO segmentation format:
    <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
    
    Coordinates are normalized to [0, 1] based on the image dimensions.
    """
    if "runwayLabel" not in data:
        raise KeyError("'runwayLabel' key not found in data dict.")
    
    polygon = ensure_clockwise_quad(data["runwayLabel"])
    width, height = image.size

    # Normalize coordinates to 0-1 range
    normalized_points = [(pt[0] / width, pt[1] / height) for pt in polygon]
    
    flat_coords = " ".join(f"{x:.6f} {y:.6f}" for x, y in normalized_points)
    return f"{class_id} {flat_coords}"


def create_runway_segmentation_mask(image: Image.Image, data: dict) -> Image.Image:
    if "runwayLabel" not in data:
        raise KeyError("'runwayLabel' key not found in data dict.")
    
    polygon = ensure_clockwise_quad(data["runwayLabel"])
    width, height = image.size
    mask = Image.new("L", (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.polygon([(int(x), int(y)) for (x, y) in polygon], fill=255)

    return mask


In [15]:
from tqdm import tqdm
import os, json

def make_labels_and_masks(folder):
    valid_exts = (".jpg", ".jpeg", ".png", ".bmp")
    image_files = [f for f in os.listdir(folder) if f.lower().endswith(valid_exts) and not f.lower().endswith(".mask.png")]

    for img_name in tqdm(image_files, desc="Processing images", unit="img"):
        base_name, ext = os.path.splitext(img_name)

        json_path = os.path.join(folder, f"{base_name}.json")
        if not os.path.exists(json_path):
            print(f"Skipping {img_name}: no matching JSON {base_name}.json found.")
            continue

        with open(json_path, "r") as f:
            data = dict(json.load(f))

        image_path = os.path.join(folder, img_name)
        image = Image.open(image_path)

        yolo_label = polygon_to_yolo_label_from_image_dict(image, data)
        seg_mask = create_runway_segmentation_mask(image, data)

        label_file_path = os.path.join(folder, f"{base_name}.txt")
        with open(label_file_path, 'w', encoding='utf-8') as f:
            f.write(yolo_label)

        mask_file_path = os.path.join(folder, f"{base_name}.mask.png")
        seg_mask.save(mask_file_path)

In [16]:
make_labels_and_masks("p_BaseImages")

Processing images: 100%|██████████| 6498/6498 [00:23<00:00, 280.66img/s]


In [17]:
make_labels_and_masks("p_VariantImages")

Processing images: 100%|██████████| 19494/19494 [01:06<00:00, 295.13img/s]


In [18]:
make_labels_and_masks("p_VariantImagesWithOcclusion")

Processing images: 100%|██████████| 19494/19494 [01:06<00:00, 293.95img/s]
